In [ ]:
# Combine all data from P1, P2 into final CCRI layer

In [1]:
import os
import pandas as pd
import geopandas as gpd
import glob
import numpy as np

In [2]:
# Query data dir (avoiding hard-coding paths when working between users)
data_dir = input('Enter data directory:')
print(data_dir)

Enter data directory: /Users/kelseydoerksen/code/unicef/ccri/data


/Users/kelseydoerksen/code/unicef/ccri/data


In [158]:
# Check that data dir was input correctly
data_dir

'/Users/kelseydoerksen/code/unicef/ccri/data'

In [159]:
# Read in all the data we need
# -- exposure files
p1_exposure_file = pd.read_csv('{}/CCRI_results_misc/Merged_Exposure_Data.csv'.format(data_dir))
p2_exposure_file = pd.read_csv('{}/CCRI_results_misc/P2_Merged_Normalized_avg.csv'.format(data_dir))
p1p2_scores = pd.read_csv('{}/CCRI_results_misc/p1_p2_avg_ccri.csv'.format(data_dir))

# -- attribute files
wb_income = pd.read_csv('{}/misc/WB_INCOME.csv'.format(data_dir))
unicef_ro = pd.read_csv('{}/misc/UNICEF_PROG_REG_GLOBAL.csv'.format(data_dir))

# -- population files
childpop = pd.read_csv('{}/CCRI_results_misc/child_pop_sum_adm0.csv'.format(data_dir))
#worldpop = pd.read_csv('{}/Misc/World_Population_ByAOI_adm0.csv'.format(data_dir))

# -- boundary file
adm0 = gpd.read_file('{}/misc/adm0_boundaries_simple.geojson'.format(data_dir))

# -- fragile codes
fragile = pd.read_csv('{}/misc/List of fragile context (2025).csv'.format(data_dir))

# -- component vals
p1_components = pd.read_csv('{}/CCRI_results_misc/p1_group_mean.csv'.format(data_dir))
p2_components = pd.read_csv('{}/CCRI_results_misc/p2_group_mean.csv'.format(data_dir))

In [160]:
# Renaming some columns 
p1_exposure_file.columns = [col.replace('_absolute', '_abs_norm') if '_absolute' in col else col for col in p1_exposure_file.columns]
p1_exposure_file.columns = [col.replace('_relative', '_rel_norm') if '_relative' in col else col for col in p1_exposure_file.columns]

In [161]:
# Merge P exposures by ISO3
merged_P = (p1_exposure_file.merge(p2_exposure_file, on='iso3', how='left'))
all_P = (merged_P.merge(p1p2_scores, on='iso3', how='left'))

In [162]:
all_P = all_P.rename(columns={'iso3':'ISO3'})

In [163]:
all_P = all_P.drop(columns=['P2_arithmetic_avg_y', 'rank_reverse_x'])
all_P = all_P.rename(columns={'P2_arithmetic_avg_x': 'P2_arithmetic_avg', 'rank_reverse_y': 'rank_reverse'})

In [164]:
# Add WB income
wb_income = wb_income[['Region_Code', 'ISO3Code']]
df = (all_P.merge(wb_income, left_on='ISO3', right_on='ISO3Code', how='left').drop('ISO3Code', axis=1).rename(columns={'Region_Code': 'wb_income'}))

In [165]:
# Add Regional Office
unicef_ro = unicef_ro[['Region_Code','ISO3Code']]
df = (df.merge(unicef_ro, left_on='ISO3', right_on='ISO3Code', how='left').drop('ISO3Code', axis=1).rename(columns={'Region_Code': 'unicef_ro'}))

In [166]:
# Add population data
df_w_childpop = (df.merge(childpop, how='left').rename(columns={'child_population': 'u18_pop'})).drop(columns=['child_population_gpw'])
df_w_childpop['u18_pop'] = df_w_childpop['u18_pop'].astype(int)

In [167]:
'''
# Using World_Population_ByAOI_adm0.csv file -> removing for now to use the population data from the exposure files to keep consistent
df_w_allpop = (df_w_childpop.merge(worldpop, left_on='name', right_on='adm0_name', how='left').rename(columns={'sum': 'total_pop'})).drop(columns=['iso3','adm0_name'])

# Set population data as integer
df_w_allpop['u18_pop'] = df_w_allpop['u18_pop'].astype(int)
df_w_allpop['total_pop'] = df_w_allpop['total_pop'].astype(int)
'''

"\n# Using World_Population_ByAOI_adm0.csv file -> removing for now to use the population data from the exposure files to keep consistent\ndf_w_allpop = (df_w_childpop.merge(worldpop, left_on='name', right_on='adm0_name', how='left').rename(columns={'sum': 'total_pop'})).drop(columns=['iso3','adm0_name'])\n\n# Set population data as integer\ndf_w_allpop['u18_pop'] = df_w_allpop['u18_pop'].astype(int)\ndf_w_allpop['total_pop'] = df_w_allpop['total_pop'].astype(int)\n"

In [168]:
# Using simplified boundaries for geometry 
adm0 = adm0[['ISO3', 'name', 'ucode','uuid','geometry','type']]
df_combined = (df_w_childpop.merge(adm0, on=['ISO3', 'name'], how='left'))

In [169]:
# Grabbing actual exposure numbers
# Define file paths
exposure_path = "{}/p1_exposure".format(data_dir)

# Get all CSV files for exposure
exposure_files = glob.glob(os.path.join(exposure_path, "*.csv"))

# Initialize empty list for processed data
exposure_data_list = []

### **Process Each File in One Loop**
for file in exposure_files:
    df = pd.read_csv(file)  # Read full file to check available columns
    filename_only = os.path.basename(file)
    hazard_name = '_'.join(filename_only.split('_')[:2])  # Extract hazard name

    # Ensure required columns exist
    required_cols = {'iso3', 'adm0_name', 'child_population_exposed'}
    if not required_cols.issubset(df.columns):
        print(f"Skipping {file}: Missing columns {required_cols - set(df.columns)}")
        continue  # Skip if required columns are missing

    df.dropna(subset=['child_population_exposed'], inplace=True)

    # Compute relative exposure (%)
    df['{}_rel'.format(hazard_name)] = np.where(
        (df['child_population_total'] > 0) & (~df['child_population_total'].isna()),
        (df['child_population_exposed'] / df['child_population_total']) * 100,
        0
    )

    # Rename to hazard
    df = df.rename(columns={'child_population_exposed': '{}_abs'.format(hazard_name)})
    df = df.drop(columns=['child_population_total']) 
    
    exposure_data_list.append(df)

In [170]:
merged_exposure_df = pd.concat(exposure_data_list, axis=1).drop_duplicates(subset=['iso3', 'adm0_name'])
# Ensure no duplicate columns before merging
merged_exposure_df = merged_exposure_df.loc[:, ~merged_exposure_df.columns.duplicated()]

In [171]:
df_combined = (df_combined.merge(merged_exposure_df, left_on=['ISO3', 'adm0_name'], right_on=['iso3', 'adm0_name'], how='left'))

In [172]:
df_combined = df_combined.rename(columns={'population_total': 'total_pop'})

In [173]:
# --- Adding P2 data
vul_path = "{}/p2_vulnerability".format(data_dir)
total_population_file = "{}/CCRI_results_misc/child_pop_sum_adm0.csv".format(data_dir)

# Load total child population data
total_pop_df = pd.read_csv(total_population_file, usecols=['ISO3', 'name', 'child_population'])

# Ensure unique ISO3-name pairs before merging
total_pop_df = total_pop_df.groupby(['ISO3', 'name'], as_index=False).agg({'child_population': 'mean'})

# Get all CSV files for exposure
p2_vul_files = glob.glob(os.path.join(vul_path, "*.csv"))

# Initialize empty list for processed data
vul_data_list = []

for file in p2_vul_files:
    df = pd.read_csv(file)  # Read full file to check available columns
    if 'iso3' not in df.columns or 'value' not in df.columns:
        continue  # Skip files missing required columns
    # Normalize 'value' column
    df = df[['iso3', 'value']].dropna()
    df = df.rename(columns={'iso3': 'ISO3'}, errors='ignore')
    filename_only = os.path.basename(file)
    hazard_name = '_'.join(filename_only.split('.csv')[:1])  # Extract hazard name
    if hazard_name == 'P2_Child_Mortality':
        continue

    # merge with population data
    df = (df.merge(total_pop_df, on='ISO3', how='left'))

    # Compute total and relative vulnerability
    df['{}'.format(hazard_name)] = np.where(
        (df['child_population'] > 0) & (~df['child_population'].isna()),
        (df['value']),
        0
    )

    # Rename for relative
    df = df.drop(columns=['child_population', 'time_period', 'data_source', 'value'], errors='ignore')
    
    vul_data_list.append(df)

In [174]:
for i in range(len(vul_data_list)):
    df_combined = (df_combined.merge(vul_data_list[i], on=['ISO3', 'name'], how='left'))

In [175]:
# Add fragile
fragile['fragile'] = 'fragile'
df_combined = (df_combined.merge(fragile[['ISO3','fragile']], on=['ISO3'], how='left'))

In [182]:
df_combined = df_combined.drop(columns=['iso3'])

In [183]:
# Add components
# Rename
p1_components = p1_components.rename(columns={'river_flood_gmean': 'P1_rfl', 'coastal_flood_gmean': 'P1_cfl',
                                             'storm_gmean': 'P1_ts', 'drought_gmean': 'P1_dr', 'heat_gmean': 'P1_hw',
                                             'fire_gmean': 'P1_fr', 'sand_dust_gmean': 'P1_sds', 'air_pollution_gmean': 'P1_pm25',
                                             'malaria_gmean': 'P1_mal'})

p2_components = p2_components.rename(columns={'health': 'P2_hea', 'nutrition': 'P2_nut', 'education': 'P2_edu',
                                             'protection': 'P2_pro', 'poverty': 'P2_pov', 'survival': 'P2_sur'})

In [184]:
df_combined = (df_combined.merge(p1_components, left_on=['ISO3', 'adm0_name'], right_on=['iso3', 'adm0_name'], how='left'))

In [185]:
df_combined = (df_combined.merge(p2_components, left_on=['ISO3'], right_on=['iso3'], how='left'))

In [186]:
df_combined = df_combined.drop(columns=['adm0_name'])

In [187]:
# Remove any columns we don't want and rearrange as well
# -- Dropping
# Drop min, max
df_combined = df_combined.drop(df_combined.filter(regex='max').columns, axis=1)
df_combined = df_combined.drop(df_combined.filter(regex='min').columns, axis=1)

# -- Renaming
df_combined = df_combined.rename(columns={'name': 'adm_name', 'P1_P2_geometric_avg': 'ccri', 'ISO3':'iso3'})

In [188]:
# Set to 2 decimal places
for col in df_combined.columns:
    if type(df_combined['{}'.format(col)].iloc[0]) != str:
        if col in ['wb_income', 'unicef_ro', 'geometry', 'fragile']:
            continue
        else:
            df_combined[col] = df_combined[col].round(2)

In [189]:
# Population to integer
df_combined['u18_pop'] = df_combined['u18_pop'].astype(int)
df_combined['total_pop'] = df_combined['total_pop'].astype(int)

In [190]:
# Rename normalized to _norm
df_combined.columns = [col.replace('_normalized', '_norm') if 'normalized' in col else col for col in df_combined.columns]

In [191]:
list(df_combined.columns)

['iso3',
 'river_flood_abs_norm',
 'river_flood_rel_norm',
 'coastal_flood_abs_norm',
 'coastal_flood_rel_norm',
 'tropical_storm_abs_norm',
 'tropical_storm_rel_norm',
 'agricultural_drought_abs_norm',
 'agricultural_drought_rel_norm',
 'drought_sma_abs_norm',
 'drought_sma_rel_norm',
 'drought_spi_abs_norm',
 'drought_spi_rel_norm',
 'heatwave_frequency_abs_norm',
 'heatwave_frequency_rel_norm',
 'heatwave_duration_abs_norm',
 'heatwave_duration_rel_norm',
 'heatwave_severity_abs_norm',
 'heatwave_severity_rel_norm',
 'extreme_heat_abs_norm',
 'extreme_heat_rel_norm',
 'fire_FRP_abs_norm',
 'fire_FRP_rel_norm',
 'fire_frequency_abs_norm',
 'fire_frequency_rel_norm',
 'sand_dust_abs_norm',
 'sand_dust_rel_norm',
 'air_pollution_abs_norm',
 'air_pollution_rel_norm',
 'vectorborne_malariapv_abs_norm',
 'vectorborne_malariapv_rel_norm',
 'vectorborne_malariapf_abs_norm',
 'vectorborne_malariapf_rel_norm',
 'P2_WASH_Drinking_Water_value_norm',
 'P2_WASH_Sanitation_value_norm',
 'P2_LSCED_

In [192]:
# Renaming the hazards
# P1 hazards
df_combined.columns = [col.replace('river_flood', 'rfl') if 'river_flood' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('coastal_flood', 'cfl') if 'coastal_flood' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('tropical_storm', 'ts') if 'tropical_storm' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('agricultural_drought', 'agdr') if 'agricultural_drought' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('drought_sma', 'metdr_sma') if 'drought_sma' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('drought_spi', 'metdr_spi') if 'drought_spi' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('heatwave_frequency', 'hw_fre') if 'heatwave_frequency' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('heatwave_severity', 'hw_sev') if 'heatwave_severity' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('heatwave_duration', 'hw_dur') if 'heatwave_duration' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('extreme_heat', 'ext') if 'extreme_heat' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('fire_frequency', 'fr_fre') if 'fire_frequency' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('fire_FRP', 'fr_int') if 'fire_FRP' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('sand_dust', 'sds') if 'sand_dust' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('vectorborne_malariapv', 'mal_pv') if 'vectorborne_malariapv' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('vectorborne_malariapf', 'mal_pf') if 'vectorborne_malariapf' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('air_pollution', 'pm25') if 'air_pollution' in col else col for col in df_combined.columns]

# P2 hazards
df_combined.columns = [col.replace('P2_Immunization_DTP1', 'hea_dtp1') if 'P2_Immunization_DTP1' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Immunization_DTP3', 'hea_dtp3') if 'P2_Immunization_DTP3' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Birth_Attendant_Y15T19', 'hea_skat') if 'P2_Birth_Attendant_Y15T19' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_electricity_access', 'hea_elec') if 'P2_electricity_access' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Nutrition_Stunting_Modeled', 'nut_stu') if 'P2_Nutrition_Stunting_Modeled' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_food_poverty', 'nut_fpov') if 'P2_food_poverty' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_WASH_Drinking_Water', 'wash_wat') if 'P2_WASH_Drinking_Water' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_WASH_Drinking_Sanitation', 'wash_san') if 'P2_WASH_Drinking_Sanitation' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_WASH_Sanitation', 'wash_san') if 'P2_WASH_Sanitation' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_basic_hygiene', 'wash_hyg') if 'P2_basic_hygiene' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_LSCED', 'edu_lsos') if 'P2_LSCED' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_ED_CR_L2', 'edu_lscr') if 'P2_ED_CR_L2' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Learning_Poverty', 'edu_lpov') if 'P2_Learning_Poverty' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_PT_Labor', 'pro_lab') if 'P2_PT_Labor' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Child_Marriage', 'pro_mar') if 'P2_Child_Marriage' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Child_poverty', 'prov_md') if 'P2_Child_poverty' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Social_Protection', 'prov_u5sp') if 'P2_Social_Protection' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Under5_Mortality', 'sur_u5mor') if 'P2_Under5_Mortality' in col else col for col in df_combined.columns]

In [193]:
df_combined.columns = [col.replace('value_norm', 'norm') if 'value_norm' in col else col for col in df_combined.columns]

In [194]:
list(df_combined.columns)

['iso3',
 'rfl_abs_norm',
 'rfl_rel_norm',
 'cfl_abs_norm',
 'cfl_rel_norm',
 'ts_abs_norm',
 'ts_rel_norm',
 'agdr_abs_norm',
 'agdr_rel_norm',
 'metdr_sma_abs_norm',
 'metdr_sma_rel_norm',
 'metdr_spi_abs_norm',
 'metdr_spi_rel_norm',
 'hw_fre_abs_norm',
 'hw_fre_rel_norm',
 'hw_dur_abs_norm',
 'hw_dur_rel_norm',
 'hw_sev_abs_norm',
 'hw_sev_rel_norm',
 'ext_abs_norm',
 'ext_rel_norm',
 'fr_int_abs_norm',
 'fr_int_rel_norm',
 'fr_fre_abs_norm',
 'fr_fre_rel_norm',
 'sds_abs_norm',
 'sds_rel_norm',
 'pm25_abs_norm',
 'pm25_rel_norm',
 'mal_pv_abs_norm',
 'mal_pv_rel_norm',
 'mal_pf_abs_norm',
 'mal_pf_rel_norm',
 'wash_wat_norm',
 'wash_san_norm',
 'edu_lsos_norm',
 'nut_stu_norm',
 'P2_Child_Mortality_norm',
 'hea_dtp1_norm',
 'hea_dtp3_norm',
 'pro_lab_norm',
 'edu_lpov_norm',
 'edu_lscr_norm',
 'hea_skat_norm',
 'prov_md_norm',
 'pro_mar_norm',
 'nut_fpov_norm',
 'prov_u5sp_norm',
 'wash_hyg_norm',
 'hea_elec_norm',
 'P2_arithmetic_avg',
 'P1_geometric_avg',
 'ccri',
 'rank_reverse

In [195]:
df_combined = df_combined[[
    'iso3',
    'adm_name',
    'total_pop',
    'u18_pop',
    'wb_income',
    'unicef_ro',
    'ucode',
    'uuid',
    'geometry',
    'type',
    'fragile','rfl_abs',
 'rfl_rel','rfl_abs_norm',
 'rfl_rel_norm','cfl_abs',
 'cfl_rel','cfl_abs_norm',
 'cfl_rel_norm','ts_abs',
 'ts_rel','ts_abs_norm',
 'ts_rel_norm','agdr_abs',
 'agdr_rel','agdr_abs_norm',
 'agdr_rel_norm','metdr_sma_abs',
 'metdr_sma_rel','metdr_sma_abs_norm',
 'metdr_sma_rel_norm','metdr_spi_abs',
 'metdr_spi_rel','metdr_spi_abs_norm',
 'metdr_spi_rel_norm','hw_fre_abs',
 'hw_fre_rel','hw_fre_abs_norm',
 'hw_fre_rel_norm','hw_dur_abs',
 'hw_dur_rel','hw_dur_abs_norm',
 'hw_dur_rel_norm','hw_sev_abs',
 'hw_sev_rel','hw_sev_abs_norm',
 'hw_sev_rel_norm','ext_abs',
 'ext_rel','ext_abs_norm',
 'ext_rel_norm','fr_fre_abs',
 'fr_fre_rel','fr_fre_abs_norm',
 'fr_fre_rel_norm','fr_int_abs',
 'fr_int_rel','fr_int_abs_norm',
 'fr_int_rel_norm','sds_abs',
 'sds_rel','sds_abs_norm',
 'sds_rel_norm','pm25_abs',
 'pm25_rel','pm25_abs_norm',
 'pm25_rel_norm','mal_pv_abs',
 'mal_pv_rel','mal_pv_abs_norm',
 'mal_pv_rel_norm','mal_pf_abs',
 'mal_pf_rel','mal_pf_abs_norm',
 'mal_pf_rel_norm','hea_dtp1','hea_dtp1_norm', 'hea_dtp3','hea_dtp3_norm','hea_skat','hea_skat_norm',
'hea_elec','hea_elec_norm','nut_stu','nut_stu_norm','nut_fpov','nut_fpov_norm','wash_wat',
    'wash_wat_norm','wash_san','wash_san_norm','wash_hyg','wash_hyg_norm','edu_lsos','edu_lsos_norm',
    'edu_lscr','edu_lscr_norm','edu_lpov','edu_lpov_norm','pro_lab','pro_lab_norm','pro_mar',
    'pro_mar_norm','prov_md','prov_md_norm','prov_u5sp','prov_u5sp_norm','sur_u5mor',
    'P1_rfl', 'P1_cfl', 'P1_ts', 'P1_dr', 'P1_hw', 'P1_fr', 'P1_sds', 'P1_pm25', 'P1_mal',
    'P2_hea', 'P2_nut', 'P2_edu', 'P2_pro', 'P2_pov', 'P2_sur','P1_geometric_avg','P2_arithmetic_avg',
'ccri']]

In [196]:
# int vals that are abs 
for col in df_combined.columns:
    if 'abs' in col and 'norm' not in col:
        df_combined[col] = df_combined[col].astype(int)

In [197]:
# Adding in component column, first let's make a dictionary
component_dict = {
    'Riverine Floods': ['rfl'],
    'Coastal Floods': ['cfl'],
    'Tropical Storms': ['ts'],
    'Droughts': ['agdr', 'metdr'],
    'Heatwave': ['hw'],
    'Extreme Heat': ['ext'],
    'Fire': ['fr'],
    'Sand and Dust Storm': ['sds'],
    'Air pollution': ['pm25'],
    'Vector Borne Diseases': ['mal'],
    'Health': ['hea'],
    'Nutrition': ['nut'],
    'WASH': ['wash'],
    'Education': ['edu'],
    'Protection': ['pro'],
    'Poverty': ['pov'],
    'Child Survival': ['sur']
}

In [198]:
gdf = gpd.GeoDataFrame(df_combined, geometry=df_combined['geometry'], crs='EPSG:4326')

In [200]:
gdf.to_file('{}/CCRI_P1_P2_format_10042025.geojson'.format(data_dir))